In [1]:
import sys
sys.path.insert(0, '/home/xhding/workspaces/torch-deep-retina')
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
import torch
from kinetic.evaluation import *
from kinetic.utils import select_model
from kinetic.config import get_custom_cfg
from kinetic.data import TestDataset

In [2]:
device = torch.device('cuda:1')
model = KineticsChannelModel().to(device)
cfg = get_custom_cfg('channel')
model = select_model(cfg, device)
checkpoint_path = '/home/xhding/saved_model/channel/epoch_080_loss_-3.33_pearson_0.4375.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

KineticsChannelModel(
  (bipolar): Sequential(
    (0): LinearStackedConv2d(
      bias=True, abs_bnorm=False
      (convs): Sequential(
        (0): Conv2d(40, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (4): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (5): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (6): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1))
      )
    )
    (1): Flatten()
    (2): BatchNorm1d(10368, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (3): GaussianNoise(std=0.05, trainable=False, adapt=False, momentum=0.95)
    (4): Softplus(beta=1, threshold=20)
    (5): Clamp()
    (6): Multiply()
    (7): Reshape(shape=(-1, 8, 1296))
  )
  (kinetics): Kinetics_channel(dt=0.001)

In [5]:
test_data =  DataLoader(TestDataset(cfg), batch_size=1)
test_pc = pearsonr_eval(model, test_data, 5, device)
test_pc

0.5706678715312947

In [6]:
test_data =  DataLoader(TestDataset(cfg), batch_size=1, shuffle=True)
test_pc = pearsonr_eval(model, test_data, 5, device)
test_pc

0.15041225536943686